In [19]:
import telebot
from telebot import types
import sqlite3 as sq
import datetime
import re
#import const

bot = telebot.TeleBot("5608244416:AAGO9fj5If9Focw1VttM2vIP7yDh7XYSRhs", parse_mode=None)

# open file with database
with sq.connect('mus_schedule.db', check_same_thread=False) as connect:
  cursor = connect.cursor()

# create date: 05.03.2023 and time: 12:00
date_1 = datetime.date(2023, 3, 5)
time_1 = datetime.time(12,0)

# create an empty dictionary to store user data
user_data = {}

cursor.execute(f"SELECT SUM(quantity) FROM {table_name}")
total_quantity = cursor.fetchone()[0]

if total_quantity is not None and total_quantity > 50:
  # create new table with group for the next hour
  datetime_2 = datetime.datetime.combine(date_1, time_1) + datetime.timedelta(hours=1) #combine to convert in 'date' object 
  time_2 = datetime_2.time() #cause adding hours is forbidden 
  table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_2.strftime('%H_%M')}"
    
  cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
            name STRING,
            quantity INTEGER,
            contact STRING UNIQUE
            );   """)
  connect.commit()

table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_1.strftime('%H_%M')}"#.replace('-', '_')
#SQL doesn`t allow '-', so i have to replace it

@bot.message_handler(commands=['omet','start']) # comands that responded
#@bot.message_handler(content_types=['text','sticker'])
def respond(message):
  
  global table_name
  
  
  
  
    
    # photo_mus = types.InputMediaPhoto('https://scontent-ams2-1.xx.fbcdn.net/v/t1.6435-9/193684250_1184653448716244_1549048136683557681_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=8bfeb9&_nc_ohc=7yIjZg512oIAX-92GYU&_nc_ht=scontent-ams2-1.xx&oh=00_AfAudupE6CtYIyfwv4wymPOXc7HrylZBNvlL-KIs7B_Q3Q&oe=643A8642')
    
  text_menu=f"<b>{message.from_user.first_name}</b>.,виберіть та натисніть одну з кнопок нижче\n\
           1.Зареєструватись на найближчу екскурсію.\n\
           2.Відвідати наш сайт.\n\
           3.Залишити відгук відносно роботи електротранспорту."
  markup=types.InlineKeyboardMarkup(row_width=1) #, resize_keyboard=True)
    
      
  button_1=types.InlineKeyboardButton(f"1.Екскурсія\
    {date_1.strftime ('%d-%m-%Y')} о {time_1}",callback_data='schedule')
     #photo = photo.read(),\
      #photo = photo_mus)
  with open('photo_2023-03-13_10-20-36.jpg', 'rb') as photo:
    bot.send_photo(message.chat.id, photo, reply_markup=markup)
  button_2=types.InlineKeyboardButton("2.Відвідати офіційну сторінку КП 'OMET'",\
    url="https://oget.od.ua",callback_data='site')
  button_3=types.InlineKeyboardButton("3.Залишити відгук.", callback_data='respond')
  markup.add(button_1,button_2,button_3)
    
  bot.send_message(message.chat.id,text_menu,reply_markup=markup, parse_mode='html') #,resize_keybord=True)



@bot.callback_query_handler(func=lambda call:True)
def callback(call):
  if call.message: # check if button pushed
    """Виклик bot.register_next_step_handler(call.message, save_quantity)
    в callback функції необхідно перенести в середину умови if call.data == 'schedule'."""
    if call.data == 'schedule':
      bot.register_next_step_handler(call.message, save_quantity)
      bot.send_message(call.message.chat.id, "Напишіть будь ласка цифрами (від 1 до 50) кількість людей,\
              які планують відвідати музей КП 'ОМЕТ'\n\
              (наприклад:'12' - бот ідентифікує лише цифрову складову у цьому повідомленні)")
  



def save_quantity(message):
  quantity = message.text.strip()  # remove extra spaces from text 
  
  user_data['name'] = message.from_user.first_name
    
  # check if quantity is real number
  if not quantity.isdigit():
    bot.send_message(message.chat.id, "Невірно введена кількість")
    bot.register_next_step_handler(message, save_quantity)  # waiting for next message
    return
  
  # check if quantity more than 50
  if int(quantity) > 50:
    bot.send_message(message.chat.id, "Зв'яжіться із директором музею за телефоном 050 399 42 11 для обговорення окремої\
      екскурсії для такої великої кількості людей\n\
      Якщо хочете повернутись в головне меню напишіть команду '/start' або '/omet'")
    return
  
  # separate the numerical part from the text, if there is one
  digits_str = ''.join(re.findall('\d+', quantity))
  digits_int = int(digits_str) if digits_str else None
  
  user_data['quantity'] = digits_int
  
  bot.send_message(message.chat.id, f"Кількість відвідувачів {digits_int} збережено")
  
  bot.send_message(message.chat.id, "Надішліть будь ласка ваш номер телефону для зв'язку з вами,\n\
    наприклад: 0661234567 - , без пробілів та інших символів крім цифр")
  
  bot.register_next_step_handler(message, ask_phone)
                   


def ask_phone(message):
  global table_name
  phone_number = message.text.strip()  # remove extra spaces from text
  int_phone = int(''.join(re.findall('\d+', phone_number)))
  
  # check if phone number is valid
  #if not phone_number.isdigit() or not 6 <= len(phone_number) <= 13:
  if not re.match(r'^\+?\d{10,13}$', phone_number):
    bot.send_message(message.chat.id, "В номері невідповідна кількість цифр або він містить недопустимі символи")
    bot.register_next_step_handler(message, ask_phone)  # waiting for next message
    return
  
  user_data['phone'] = int_phone
  
  # open file with database
  with sq.connect('mus_schedule.db', check_same_thread=False) as connect:
    cursor = connect.cursor()
      
    cursor.execute(f"Insert Or Replace Into {table_name} (name, quantity, contact) VALUES (?, ?, ?)",\
      (user_data['name'], user_data['quantity'], user_data['phone']))
    connect.commit()
  
    connect.close()

  bot.send_message(message.chat.id, f"Ваш контакт збережено. Будемо раді бачити вас у музеї, гарного дня!\
    Якщо хочете повернутись в головне меню натисніть-напишіть команду '/start' або '/omet'")
  
  #bot.register_next_step_handler(message, respond)

  bot.register_next_step_handler(message, respond)

bot.infinity_polling()

2023-03-17 16:57:02,474 (__init__.py:960 MainThread) ERROR - TeleBot: "Infinity polling exception: Cannot operate on a closed database."
2023-03-17 16:57:02,479 (__init__.py:962 MainThread) ERROR - TeleBot: "Exception traceback:
sqlite3.ProgrammingError: Cannot operate on a closed database.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Python\lib\site-packages\telebot\__init__.py", line 955, in infinity_polling
    self.polling(non_stop=True, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "c:\Python\lib\site-packages\telebot\__init__.py", line 1043, in polling
    self.__threaded_polling(non_stop=non_stop, interval=interval, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "c:\Python\lib\site-packages\telebot\__init__.py", line 1118, in __threaded_polling
    raise e
  File "c:\Python\lib\site-packages\telebot\__init__.py", line 1074, in __threaded_polling
    self.worker_pool.ra

In [20]:
with sq.connect('mus_schedule.db', check_same_thread=False) as connect:
  cursor = connect.cursor()



# create date: 05.03.2023 and time: 12:00
date_1 = datetime.date(2023, 3, 5)
time_1 = datetime.time(12,0)

table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_1.strftime('%H_%M')}"#.replace('-', '_')

user_data = {}

cursor.execute(f"SELECT SUM(quantity) FROM {table_name}")
total_quantity = cursor.fetchone()[0]


if total_quantity is not None and total_quantity > 50:
  # create new table with group for the next hour
  datetime_2 = datetime.datetime.combine(date_1, time_1) + datetime.timedelta(hours=1) #combine to convert in 'date' object 
  time_2 = datetime_2.time() #cause adding hours is forbidden 
  table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_2.strftime('%H_%M')}"
  
  def create_table_if_not_exists(connect, table_name):
    cursor = connect.cursor()
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
        name STRING,
        quantity INTEGER,
        contact STRING UNIQUE
        );""")
    connect.commit()
  
  cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
            name STRING,
            quantity INTEGER,
            contact STRING UNIQUE
            );   """)
  connect.commit()


In [ ]:
# Отримуємо кількість записаних людей у поточній таблиці
cursor.execute(f"SELECT SUM(quantuty) FROM {table_name}")
current_count = cursor.fetchone()[0]

if current_count + user_data['quantity'] > 50:
    # Якщо кількість записів перевищує 50, створюємо нову таблицю з наступним часом екскурсії
    next_time = datetime.datetime.combine(date_1, time_1) + datetime.timedelta(hours=1)
    next_table_name = f"tour_at_{next_time.strftime('%d_%m_%Y')}_{next_time.strftime('%H_%M')}"
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {next_table_name} (
        name STRING,
        quantity INTEGER,
        contact STRING UNIQUE
        );""")
    connect.commit()
    
    # Записуємо дані у нову таблицю
    cursor.execute(f"INSERT INTO {next_table_name} (name, quantity, contact) VALUES (?, ?, ?)",
                   (user_data['name'], user_data['quantity'], user_data['phone']))
    connect.commit()
    bot.send_message(message.chat.id, f"Кількість записів досягла 50. Ви записані на наступну екскурсію о {next_time.strftime('%H:%M')}")

else:
    # Інакше записуємо дані у поточну таблицю
    cursor.execute(f"INSERT INTO {table_name} (name, quantity, contact) VALUES (?, ?, ?)",
                   (user_data['name'], user_data['quantity'], user_data['phone']))
    connect.commit()
    bot.send_message(message.chat.id, f"Ваш контакт збережено. Будемо раді бачити вас у музеї, гарного дня!")

12:00:00 13:00:00


In [21]:
import re

text = "3 людей"
list_numbers = re.findall(r'\d+', text)

print(list_numbers)

text = "23 особи"

# Знаходимо всі цифри у тексті та зберігаємо їх як рядок
digits_str = ''.join(re.findall('\d+', text))

# Перетворюємо рядок з цифрами на integer
digits_int = int(digits_str)

print(digits_int)  # виведе 23

phone_number = '21456'

"""^ означає початок рядка
\+? перевіряє, чи присутній символ "+", який може бути або немає в номері телефону
\d{6,13} перевіряє, чи присутні в рядку рівно 6-13 цифр
$ означає кінець рядка"""

re.match(r'^\+?\d{6,13}$', phone_number)

['3']
23


In [11]:
import datetime

"""Зверніть увагу, що я змінив деякі частини вашого коду:
datetime.datetime.now().time() замість datetime.now().time(), щоб отримати поточний час з датою.
datetime.timedelta замість просто timedelta, оскільки ви імпортували timedelta з модуля datetime,
але не з модуля datetime.datetime.
datetime.datetime.combine(datetime.date.today(), current_time) замість datetime.combine(datetime.min,
current_time), оскільки datetime.min містить дату 1 січня 1 року, а не поточну дату, тому
datetime.datetime.combine(datetime.min, current_time) може дати неправильні результати при додаванні до
поточного часу."""

import datetime

time_1 = datetime.time(12, 0)  # 12:00 PM, наприклад
today = datetime.date.today()  # поточна дата
datetime_1 = datetime.datetime.combine(today, time_1)  # комбінуємо дату та час
datetime_2 = datetime_1 + datetime.timedelta(hours=1)  # додаємо одну годину
time_2 = datetime_2.time()  # отримуємо час з datetime_2

print (time_1, time_2)

# створюємо дату 20 лютого 2023 року
date = datetime.date(2023, 2, 20)

# створюємо час 13:30:00
time = datetime.time(13, 30, 0)

# створюємо дату та час 20 лютого 2023 року о 13:30:00
datetime_obj = datetime.datetime(2023, 2, 20, 13, 30, 0)

#щоб отримати поточну дату та час
now = datetime.datetime.now()

#щоб отримати час через годину від поточного моменту
current_time = datetime.datetime.now().time()  # поточний час
delta_hours = datetime.timedelta(hours=1)  # різниця у 1 години
new_time = (datetime.datetime.combine(datetime.date.today(), current_time) + delta_hours).time() # новий час

print('new_time: ', new_time)

print('now: ',now)

print('date: ', date)
print('time: ', time)
print('datetime: ', datetime_obj)

date_1 = datetime.date(2023, 3, 5)
formatted_date = date_1.strftime('%d, %m, %Y')

print (date_1)
print(formatted_date)  # виведе '05, 03, 2023'


12:00:00 13:00:00
new_time:  17:30:52.718186
now:  2023-03-17 16:30:52.718186
date:  2023-02-20
time:  13:30:00
datetime:  2023-02-20 13:30:00
2023-03-05
05, 03, 2023


In [ ]:
import telebot
from telebot import types
import sqlite3 as sq
import datetime
import re

bot = telebot.TeleBot("5608244416:AAGO9fj5If9Focw1VttM2vIP7yDh7XYSRhs", parse_mode=None)

# create date: 05.03.2023 and time: 12:00
date_1 = datetime.date(2023, 3, 5)
time_1 = datetime.time(12,0)

table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_1.strftime('%H_%M')}"#.replace('-', '_')
#SQL doesn`t allow '-', so i have to replace it

# create an empty dictionary to store user data
user_data = {}


@bot.message_handler(commands=['omet','start']) # comands that responded
# @bot.message_handler(content_types=['text','sticker'])
def respond(message):
    
    # open file with database
    connect = sq.connect('mus_schedule.db')
    cursor = connect.cursor()
    
    cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
            name STRING,
            quantity INTEGER,
            contact STRING UNIQUE
            );   """)
    connect.commit()
    
    text_menu=f"<b>{message.from_user.first_name}</b>.,виберіть та натисніть одну з кнопок нижче\n\
           1.Зареєструватись на найближчу екскурсію.\n\
           2.Відвідати наш сайт.\n\
           3.Залишити відгук відносно роботи електротранспорту."
    markup=types.InlineKeyboardMarkup(row_width=1) 
    button_1=types.InlineKeyboardButton(f"1.Зареєструватись на екскурсія\
      {date_1.strftime ('%d-%m-%Y')} о {time_1}",callback_data='schedule')
    button_2=types.InlineKeyboardButton("2.Відвідати офіційну сторінку КП 'OMET'",\
      url="https://oget.od.ua",callback_data='site')
    button_3=types.InlineKeyboardButton("3.Залишити відгук відносно роботи електротранспорту.",\
      callback_data='respond')
    markup.add(button_1,button_2,button_3)
    
    bot.send_message(message.chat.id,text_menu,reply_markup=markup,parse_mode='html')



@bot.callback_query_handler(func=lambda call:True)
def callback(call):
  if call.message: # check if button pushed
    """Виклик bot.register_next_step_handler(call.message, save_quantity)
    в callback функції необхідно перенести в середину умови if call.data == 'schedule'."""
    if call.data == 'schedule':
      bot.register_next_step_handler(call.message, save_quantity)
      bot.send_message(call.message.chat.id, "Напишіть будь ласка цифрами (від 1 до 50) кількість людей,\
              які планують відвідати музей КП 'ОМЕТ'\n\
              (наприклад:'12' - бот ідентифікує лише цифрову складову у цьому повідомленні)")
  



def save_quantity(message):
  quantity = message.text.strip()  # remove extra spaces from text 
  
  user_data['name'] = message.from_user.first_name
    
  # check if quantity is real number
  if not quantity.isdigit():
    bot.send_message(message.chat.id, "Невірно введена кількість")
    bot.register_next_step_handler(message, save_quantity)  # waiting for next message
    return
  
  # check if quantity more than 50
  if int(quantity) > 50:
    bot.send_message(message.chat.id, "Зв'яжіться із директором музею за телефоном 050 399 42 11 для обговорення окремої\
      екскурсії для такої великої кількості людей\n\
      Якщо хочете повернутись в головне меню напишіть команду '/start' або '/omet'")
    return
  
  # separate the numerical part from the text, if there is one
  digits_str = ''.join(re.findall('\d+', quantity))
  digits_int = int(digits_str) if digits_str else None
  
  user_data['quantity'] = digits_int
  
  bot.send_message(message.chat.id, f"Кількість відвідувачів {digits_int} збережено")
  
  bot.send_message(message.chat.id, "Надішліть будь ласка ваш номер телефону для зв'язку з вами:")
  
  bot.register_next_step_handler(message, ask_phone)
                   


def ask_phone(message):
  phone_number = message.text.strip()  # remove extra spaces from text
  int_phone = int(''.join(re.findall('\d+', phone_number)))
  
  # check if phone number is valid
  #if not phone_number.isdigit() or not 6 <= len(phone_number) <= 13:
  if not re.match(r'^\+?\d{10,13}$', phone_number):
    bot.send_message(message.chat.id, "В номері невідповідна кількість цифр або він містить недопустимі символи")
    bot.register_next_step_handler(message, ask_phone)  # waiting for next message
    return
  
  user_data['phone'] = int_phone
  
  # open file with database
  connect = sq.connect('mus_schedule.db')
  cursor = connect.cursor()
    
  cursor.execute(f"Insert Or Replace Into {table_name} (name, quantity, contact) VALUES (?, ?, ?)",\
    (user_data['name'], user_data['quantity'], user_data['phone']))
  connect.commit()
    
  cursor.close()
  connect.close()

  bot.send_message(message.chat.id, f"Ваш контакт збережено. Будемо раді бачити вас у музеї, гарного дня!\
    Якщо хочете повернутись в головне меню напишіть команду '/start' або '/omet'")
  
  #bot.register_next_step_handler(message, respond)


bot.infinity_polling()